# Python for Data Analysis #1 (2017-11-17)

Today we are going to cover some practical things:

* How to work use Python Excel files
* How to automate GUI tasks with Python
* Entire talk is based on [Chapters 12](https://automatetheboringstuff.com/chapter12/) and [18](https://automatetheboringstuff.com/chapter18/) from [Automate the Boring Stuff with Python: Practical programming for total beginners](https://automatetheboringstuff.com) by Al Sweigart

## Preamble

* Please start by downloading and installing v2.7.x of the **[Anaconda Python distribution](https://www.anaconda.com/download/)**
* I strongly recommend using this distribution when possible as it will **significantly decrease config and startup time**
* At this current date I **still recommened Python v2.7.x** even though version 3.x is available
  * The Python community is currently in transition, however, v2.7 is still the **more functional** of the two and is **significantly easier for beginners to use**

## Working with Excel Files: the `openpyxl` package

* We are going to use the [openpyxl](https://openpyxl.readthedocs.io/en/default/) package to work with Excel files

In [2]:
# Import required libraries

import openpyxl

### Loading Excel Files

* Excel files can be loaded with the `openpyxl.load_workbook()` command
* This returns a `Workbook` object that we can use to access sheets, etc.

In [4]:
wb = openpyxl.load_workbook("data/example.xlsx")
type(wb)

openpyxl.workbook.workbook.Workbook

### Accessing Sheets

The `Workbook` object can be used to get the individual Excel `Worksheets` through the following functions:

* `get_sheet_names()`: Returns a list of sheet names
* `get_sheet_by_name('Name'): Returns a `Worksheet` object accessing the `Name` sheet
* `active`: Returns a `Worksheet` which is currently active
* `title`: Returns the name of a Worksheet object

In [13]:
wb.get_sheet_names()

[u'Sheet1', u'Sheet2', u'Sheet3']

In [14]:
sheet = wb.get_sheet_by_name('Sheet3')

In [15]:
sheet

<Worksheet "Sheet3">

In [16]:
type(sheet)

openpyxl.worksheet.worksheet.Worksheet

In [17]:
sheet.title

u'Sheet3'

In [18]:
anotherSheet = wb.active

In [19]:
anotherSheet

<Worksheet "Sheet1">

In [20]:
print anotherSheet.title

Sheet1


### Getting Cells from the Sheets

* Entire structure is `Workbook`s contains `Worksheet`s which contain `Cell`s
* You can access the `Cell`s from a `Worksheet` object
* `Cell` objects have the `value` attribute containing the value of the cell
* Note that Python will automatically interpret the type of the cell as a Python type (e.g., `None`, `bool`, `int`, `long`, `float`, `str`, `unicode`, `tuple`, `list`, `dict`, `datetime`

In [24]:
sheet = wb.get_sheet_by_name('Sheet1')

In [25]:
sheet['A1']

<Cell Sheet1.A1>

In [26]:
sheet['A1'].value

datetime.datetime(2015, 4, 5, 13, 34, 2)

In [27]:
c = sheet['B1']
c.value

u'Apples'

In [28]:
print 'Row ' + str(c.row) + ', Column ' + c.column + ' is ' + c.value

Row 1, Column B is Apples


In [29]:
sheet['C1'].value

73L

#### Specifying columns by numbers

* Excel specifies rows by number and columns by letters (e.g., A2 is the second row and the first column)
* This actually creates a slight complexity to program, so `cell(row, column)` allows you to access specific rows and columns by number indicies

In [30]:
sheet.cell(row=1, column=2)

<Cell Sheet1.B1>

In [31]:
sheet.cell(row=1, column=2).value

u'Apples'

In [35]:
# Get every other row from the second column
for i in range(1, 8, 2):
    print i, sheet.cell(row=i, column=2).value

1 Apples
3 Pears
5 Apples
7 Strawberries


#### Determine the maximum size of the Worksheet

You can also determine the maximum size of a `Worksheet` with the `max_row` and `max_column` attributes

In [37]:
print "sheet.max_row is", sheet.max_row
print "sheet.max_column is", sheet.max_column

sheet.max_row is 7
sheet.max_column is 3


* So now you can use them programatically

In [38]:
print sheet.title, "is", sheet.max_row, "by", sheet.max_column

Sheet1 is 7 by 3


#### Might need to translate between Excel indicies (e.g., "AA" -> 27)

* Translate between string indicies to number with `cell.column_index_from_string()`
* You can also get the column from an integer with `get_column_number()`

In [39]:
from openpyxl.cell import get_column_letter, column_index_from_string

print get_column_letter(1)
print get_column_letter(27)
print column_index_from_string('A')
print column_index_from_string('AA')

A
AA
1
27


#### Getting Rows and Columns from Sheets

* Get cells of a specific column: `sheet.columns[1]` 
* Gets cells from a specific row: `sheet.row[1]`
* Get cells row-wise between diagonal coordinates: `tuple(sheet["A1":"C3"])`

In [40]:
print tuple(sheet["A1":"C3"])
sheet.columns[1]
sheet.rows[1]

((<Cell Sheet1.A1>, <Cell Sheet1.B1>, <Cell Sheet1.C1>), (<Cell Sheet1.A2>, <Cell Sheet1.B2>, <Cell Sheet1.C2>), (<Cell Sheet1.A3>, <Cell Sheet1.B3>, <Cell Sheet1.C3>))


(<Cell Sheet1.A2>, <Cell Sheet1.B2>, <Cell Sheet1.C2>)

* We can also print and get the values of each cell in a column

In [41]:
for cel in sheet.columns[1]:
    print cel.coordinate, cel.value

B1 Apples
B2 Cherries
B3 Pears
B4 Oranges
B5 Apples
B6 Bananas
B7 Strawberries


In [42]:
for cel in sheet.rows[1]:
    print cel.coordinate, cel.value

A2 2015-04-05 03:41:23
B2 Cherries
C2 85


In [43]:
for row in tuple(sheet["A1":"C3"]):
    for cel in row:
        print cel.coordinate, cel.value

A1 2015-04-05 13:34:02
B1 Apples
C1 73
A2 2015-04-05 03:41:23
B2 Cherries
C2 85
A3 2015-04-06 12:46:51
B3 Pears
C3 14


* Now it doesn't seem to exist, but we might want to translate between an index pair (x,y) to the Excel version (e.g., "XX1")

In [45]:
def create_excel_coord_str(row, col):
    if row > 0 and col > 0:
        # both row and column are greater than zero
        row_str = str(row)
        col_str = get_column_letter(col)
        ret_str = col_str + row_str
        
        return ret_str
    else:
        print "Not valid input", row, col


* Now we can access `Sheet` values by coordinate numbers

In [46]:
print create_excel_coord_str(2,3)
print create_excel_coord_str(3,3)
print tuple(sheet[create_excel_coord_str(1,1):create_excel_coord_str(3,3)])

C2
C3
((<Cell Sheet1.A1>, <Cell Sheet1.B1>, <Cell Sheet1.C1>), (<Cell Sheet1.A2>, <Cell Sheet1.B2>, <Cell Sheet1.C2>), (<Cell Sheet1.A3>, <Cell Sheet1.B3>, <Cell Sheet1.C3>))


### Activity Number 1: Census

Extract data from an Excel spreadsheet so we can process it in Python

In [47]:
wb = openpyxl.load_workbook("data/censuspopdata.xlsx")
sheet = wb.get_sheet_by_name('Population by Census Tract')

* Create a python dictionary of these data (ready for programming)

In [51]:
countyData = {}
for row in range(2, sheet.max_row + 1):
    # Each row in the spreadsheet has data for one census tract
    state = sheet['B' + str(row)].value
    county = sheet['C' + str(row)].value
    pop = sheet['D' + str(row)].value
   
    # Insert data into our data structure
    
    # Create a space for data if not there
    if state not in countyData:
        countyData[state] = {}
    if county not in countyData[state]:
        countyData[state][county] = {}
    if "tracts" not in countyData[state][county]:
        countyData[state][county]["tracts"] = 0
    if "pop" not in countyData[state][county]:
        countyData[state][county]["pop"] = 0
    
    # Add population
    countyData[state][county]["pop"] += pop
    # Increment tract
    countyData[state][county]["tracts"] += 1

* Now we can access this data...

In [52]:
print countyData['AK']['Anchorage']['pop']
print countyData['AK']['Anchorage']['tracts']

291826
55


* Or write it out

In [53]:
resultFile = open('data/census2010.txt', 'w')
resultFile.write(str(countyData))
resultFile.close()

#### Some ideas for future use here

* Read and process many many Excel files at once (provided they all have the same formatting)
* Compare data between excel spreadsheets (check for invalid data)
* Create a python program to do something with this data (enter names in a web form, submit to database, etc.)

### Creating and Saving Excel Files

Now that we know we can read from an Excel file we also create and write them:

* `openpyxl.Workbook()`: Creates a new notebook with default `Sheet` named 'Sheet'
* `sheet.title`: Can be used to get or set the title of the `Sheet`

In [54]:
wb = openpyxl.Workbook()
wb.get_sheet_names()
sheet = wb.active # only one sheet so we know its the 'active' one

In [55]:
sheet.title = 'My Report'

In [56]:
wb.save('data/my_report.xlsx')

* We can now check to see that my_report.xlsx has been created

#### Creating and removing sheets within a Worksheet

* We can create sheets with the `create_sheet(index, title)` function

In [60]:
# Creating sheets
wb.get_sheet_names()
wb.create_sheet(index=0, title='First Sheet')
wb.get_sheet_names()
wb.create_sheet(index=2, title='Middle Sheet')
wb.get_sheet_names()
wb.create_sheet(title="Last Sheet")
wb.get_sheet_names()

[u'First Sheet', u'My Report', u'Middle Sheet', u'Last Sheet', 'Last Sheet1']

* We can also remove sheets with the `remove_sheet(SheetObj)` function

In [61]:
# Removing sheets
wb.remove_sheet(wb.get_sheet_by_name('First Sheet'))
wb.remove_sheet(wb.get_sheet_by_name('Middle Sheet'))
wb.get_sheet_names()

[u'My Report', u'Last Sheet', 'Last Sheet1']

* Using the list of sheets names given to use by `get_sheet_names()` and `remove_sheet()` we can write a loop to remove all sheets from the `Workbook`

In [62]:
# Remove all sheets
for sheet_name in wb.get_sheet_names():
    wb.remove_sheet(wb.get_sheet_by_name(sheet_name))

wb.get_sheet_names()

[]

* Warning: writing out the workbook in its current state with no sheets will cause an error
* We'll add another sheet with `create_sheet()` just to be safe

In [63]:
wb.create_sheet(title="My new report")
wb.save('my_report.xlsx') # Actually errors if you try to write a workbook with no sheets

#### Writing Values to Cells

* Writing values to sheets is pretty straightforward, specify the `Cell` of a `Sheet` in square brackets and pass a value in with the assignment operator (`=`)

In [65]:
sheet = wb.active
sheet['A1'] = 'Hello world!'

#### Activity 2: Updating a Spreadsheet

* Write a program to update cells in a spreadsheet of produce sales

In [68]:
wb = openpyxl.load_workbook('data/produceSales.xlsx')
sheet = wb.get_sheet_by_name('Sheet')

price_updates = {'Garlic': 3.07,
                 'Celery': 1.19,
                 'Lemon': 1.27}

for rowNum in range(2, sheet.max_row):
    produceName = sheet.cell(row=rowNum, column=1).value
    if produceName in price_updates:
        sheet.cell(row=rowNum, column=2).value = price_updates[produceName]

wb.save('data/updatedProduceSales.xlsx')

#### Other program ideas

* Read data from one spreadsheet and write it to parts of other spreadsheets
* Read data from websites or text files write it to a spreadsheet
* Information from one spreadsheet to update another
* Automatically “clean up” data in spreadsheets. For example, it could use regular expressions to read multiple formats of phone numbers and edit them to a single, standard format.

### Setting the Font Style of Cells

Now we make are programatically created Excel `Worksheets` more presentable:

* font to set font size, color, underlining, etc.
* fill to set a pattern or color gradient
* border to set borders on a cell
* cell alignment
* set cell protection

* First I am going to create a new `Workbook` `wb2` with the workbook constructor function/method `openpyxl.Workbook()`

In [71]:
from openpyxl.styles import Font
wb2 = openpyxl.Workbook()
sheet = wb2.get_sheet_by_name('Sheet')

* Next I am going to create a `Font` object using the `Font` constructor. Note its default options are defined below

* Fonts have the following arguments

```
font = Font(name='Calibri',
            size=11,
            bold=False,
            italic=False,
            vertAlign=None,
            underline='none',
            strike=False,
            color='FF000000')
```

In [72]:
italic24Font = Font(size=24, italic=True)

* Next we are going to assign that `Font` object to the `font` attribute of a `Cell`, then add some text to the `value` attribute, and finally write out our workbook

In [73]:
sheet['A1'].font = italic24Font
sheet['A1'] = 'Hello world!'
wb2.save('data/my_formatted_spreadsheet.xlsx')

* The other thing that you can change is background color

```
fill = PatternFill(fill_type=None,
                   start_color='FFFFFFFF',
                   end_color='FF000000')
```

from openpyxl.styles import PatternFill
fill = PatternFill(fill_type="solid",
                   start_color='FF0000',
                   end_color='808080')
sheet['A1'].fill = fill
wb2.save('data/my_formatted_spreadsheet.xlsx')

* We can of course get a bit more fancy by adding color background with the `PatternFill` style

In [75]:
rainbow = ["FF0000", "FF7F00", "FFFF00", "00FF00", "0000FF", "4B0082", "9400D3"]
values = ["Richard", "Of", "York", "Gains", "Battles", "In", "Vain"]
for i, color in enumerate(rainbow):
    color_fill = PatternFill(fill_type="solid",
                             start_color=color,
                             end_color=color)
    coords = create_excel_coord_str(i+1, 2)
    print "Filling in", coords
    sheet[coords].value = values[i]
    sheet[coords].fill = color_fill
wb2.save('data/my_formatted_spreadsheet.xlsx')

Filling in B1
Filling in B2
Filling in B3
Filling in B4
Filling in B5
Filling in B6
Filling in B7


#### Homework

Play around with borders, cell alignment, and protection. There's quite a few more options to play around with `Font` and `PatternFill` too! See: https://openpyxl.readthedocs.io/en/default/styles.html

### Formulas

Strings assigned to a cell beginning with an `=` lets cell values be calculated by other cells

In [80]:
sheet['B1'].value = 1
sheet['B2'].value = 2
sheet['B3'].value = 3
sheet['B4'] = '=SUM(B1:B3)'

In [81]:
wb2.save('data/my_formulas.xlsx')

### Setting Height and Width

* You can set the column high and width

In [82]:
sheet['A1'] = 'Tall row'
sheet['B1'] = 'Wide column'
sheet.row_dimensions[1].height = 70
sheet.column_dimensions['B'].width = 140
wb2.save('data/my_formatting.xlsx')

### Merging and Unmerging Cells



In [83]:
wb = openpyxl.Workbook()
sheet = wb.active
sheet.merge_cells('A1:D3')
sheet['A1'] = 'Twelve cells merged together.'
sheet.merge_cells('C5:D5')
sheet['C5'] = 'Two merged cells.'
wb.save('data/merged.xlsx')

In [84]:
wb = openpyxl.load_workbook('data/merged.xlsx')
sheet = wb.active
sheet.unmerge_cells('A1:D3')
sheet.unmerge_cells('C5:D5')
wb.save('data/unmerged.xlsx')

### Freeze Pains

In [213]:
wb = openpyxl.load_workbook('data/produceSales.xlsx')
sheet = wb.active
sheet.freeze_panes = 'A2'
wb.save('data/freezeExample.xlsx')

### Charts

In [215]:
wb = openpyxl.Workbook()
sheet = wb.active

# create some data in column A
for i in range(1, 11):
    sheet['A' + str(i)] = i

# Create the reference object
refObj = openpyxl.chart.Reference(sheet, min_col=1, min_row=1, max_col=1, max_row=10)
# Create the series object
seriesObj = openpyxl.chart.Series(refObj, title='First series')
chartObj = openpyxl.chart.BarChart()
chartObj.title = 'My Chart'
chartObj.append(seriesObj)
sheet.add_chart(chartObj, 'C5')
wb.save('data/sampleChart.xlsx')

#### Homework

Try out `openpyxl.chart.LineChart()`, `openpyxl.chart.ScatterChart()`, and `openpyxl.chart.PieChart()`

## Controlling the Keyboard and Mouse with GUI Automation

* Going to use the `pyautogui` package to automate the use of the mouse and keyboard
* You need to install a few additional packages here: https://pyautogui.readthedocs.io/en/latest/install.html
http://pyautogui.readthedocs.io/


In [87]:
import pyautogui

### Moving the mouse

* Use the `moveTo(x, y)` function to move the mouse from its current location to its next location (x,y) measured from the top left of the screen

In [95]:
# Move our cursor to (200, 500)
pyautogui.moveTo(200, 500, duration=3)

* Move the cursor in a square three times

In [89]:
for i in range(3):
    pyautogui.moveTo(100, 100, duration=0.25)
    pyautogui.moveTo(200, 100, duration=0.25)
    pyautogui.moveTo(200, 200, duration=0.25)
    pyautogui.moveTo(100, 200, duration=0.25)

* Move the cursor relative to its current position

In [90]:
for i in range(3):
    pyautogui.moveRel(100, 0, duration=0.25)
    pyautogui.moveRel(0, 100, duration=0.25)
    pyautogui.moveRel(-100, 0, duration=0.25)
    pyautogui.moveRel(0, -100, duration=0.25)

* Use `position()` to get the current pixel coordinates of the mouse

In [93]:
pyautogui.position()

(159, 157)

### Clicking the mouse

* `click(x,y)` clicks the mouse at a x,y
* `mouseDown(x,y)` presses and holds the mouse down at x,y
* `mouseUp(x,y)` presses and holds the mouse up at x,y
* `doubleClick(x,y)` clicks twice in the current positon at x,y

Note: For all of the above, not specifying x,y will just do this at the mouse's current location

In [99]:
pyautogui.click(159, 157)
pyautogui.mouseDown()
pyautogui.mouseUp()
pyautogui.doubleClick(159,157)

### Adding a Delay

Because these actions happen so fast. It can be difficult for some GUIs to keep up. You may also want to delay so you can keep track of the work. Use the `time` package to add a delay to any program.

* `time.sleep(sec)`: causes the program to wait for a certain number of seconds

In [228]:
import time
sec=10
time.sleep(sec)

### Scrolling Up, Down, Left, and Right

* `scroll(10, x, y)`: Moves cursor to x,y an then crolls up 10 'clicks'
* `hscroll(10)`: Horizontal scroll 10 'clicks' at current position
* Note: positive values go one way, negative value go the other

In [101]:
pyautogui.scroll(10, x=100, y=100)  # move mouse cursor to 100, 200, then scroll up 10 "clicks"
pyautogui.hscroll(10)   # scroll right 10 "clicks"

### Go to the position of an image

* If you take a screenshot with an image, pyautogui, can locate it on the screen
* `locateOnScreen('path/to/image.png')`: returns the top left coordinates and offsets for the box

In [111]:
# print pyautogui.locateOnScreen('data/green2.png')
x1, y1, x_off, y_off = pyautogui.locateOnScreen('data/green2.png')
pyautogui.moveTo(x1/2, y1/2, duration=5)

In [109]:
list(pyautogui.locateAllOnScreen('data/green2.png'))

[(338, 100, 36, 32)]

In [110]:
x,y = pyautogui.center(pyautogui.locateOnScreen('data/green2.png'))
pyautogui.moveTo(x, y, duration=5)

### Write a message

* You can use `typewrite("Hello!")` to write a message on the keyboard (usually after clicking or double clicking)

In [112]:
pyautogui.position()

(664, 486)

In [113]:
pyautogui.click(664, 486)
pyautogui.typewrite('Hello world!')

* You can also specify a series of key presses by using a list() or []

In [ ]:
pyautogui.typewrite(['a', 'b', 'left', 'left', 'X', 'Y'])

### Complex Key Usage

You can use a combination of `keyDown('keyname')`, `press('keyname')`, and `keyUp('keyname')` to do more complicated shortcut commands.

* Shift + 4

In [114]:
pyautogui.keyDown('shift'); pyautogui.press('4'); pyautogui.keyUp('shift')

In [ ]:
* Ctrl + c (copy)

In [ ]:
pyautogui.keyDown('ctrl')
pyautogui.keyDown('c')
pyautogui.keyUp('c')
pyautogui.keyUp('ctrl')

* The above is common enough that there's a whole command for this.

In [115]:
pyautogui.hotkey('ctrl', 'c')

## Homework for next class

Write a script to automate something work related.